In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
from iminuit import Minuit
from iminuit.cost import LeastSquares
import pickle
import uproot as ur
import re

In [9]:
input_file = f"../../genIII_data/Sim/e+_2.5_12.5.edm4hep.root"

events = ur.open("%s:events"%(input_file))

with open("sim_geometry_12.5.pkl", 'rb') as file:
    sim_geometry = pickle.load(file)
    

even_ch_map = sim_geometry["even_ch_map"]
odd_ch_map = sim_geometry["odd_ch_map"]
z_to_layer_map = sim_geometry["z_to_layer_map"]

In [10]:
print(even_ch_map)
print(odd_ch_map)

{(-109.8, -109.8): 0, (-61.0, -109.8): 1, (-12.2, -109.8): 2, (36.6, -109.8): 3, (85.4, -109.8): 4, (-109.8, -61.0): 5, (-61.0, -61.0): 6, (-12.2, -61.0): 7, (36.6, -61.0): 8, (85.4, -61.0): 9, (-109.8, -12.2): 10, (-61.0, -12.2): 11, (-12.2, -12.2): 12, (36.6, -12.2): 13, (85.4, -12.2): 14, (-109.8, 36.6): 15, (-61.0, 36.6): 16, (-12.2, 36.6): 17, (36.6, 36.6): 18, (85.4, 36.6): 19, (-109.8, 85.4): 20, (-61.0, 85.4): 21, (-12.2, 85.4): 22, (36.6, 85.4): 23, (85.4, 85.4): 24}
{(-85.4, -109.8): 0, (-36.6, -109.8): 1, (12.2, -109.8): 2, (61.0, -109.8): 3, (85.4, -109.8): 4, (109.8, -109.8): 5, (-85.4, -85.4): 6, (-36.6, -85.4): 7, (12.2, -85.4): 8, (61.0, -85.4): 9, (85.4, -85.4): 10, (109.8, -85.4): 11, (-85.4, -36.6): 12, (-36.6, -36.6): 13, (12.2, -36.6): 14, (61.0, -36.6): 15, (85.4, -36.6): 16, (109.8, -36.6): 17, (-85.4, 12.2): 18, (-36.6, 12.2): 19, (12.2, 12.2): 20, (61.0, 12.2): 21, (85.4, 12.2): 22, (109.8, 12.2): 23, (-85.4, 61.0): 24, (-36.6, 61.0): 25, (12.2, 61.0): 26, (61.

In [11]:
num_evts = len(np.array(events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.energy"]))
all_event_energies = []

for evt in range(num_evts):
    if evt % 100 == 0:
        print(100 * evt / num_evts)
    evt_energies = []

    energies = np.array(events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.energy"])[evt]
    x_pos    = np.array(events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.position.x"])[evt]
    y_pos    = np.array(events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.position.y"])[evt]
    z_pos    = np.array(events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.position.z"])[evt]

    # Get layer numbers for each hit
    layers = np.array([z_to_layer_map[z] for z in z_pos])

    for layer in range(15):
        # Select channel map based on even/odd layer
        ch_map = even_ch_map if layer % 2 == 0 else odd_ch_map

        for ch in range(25):
            # Find (x, y) pair for this channel
            try:
                xy_lookup = [k for k, v in ch_map.items() if v == ch][0]  # (x, y)
            except IndexError:
                # If this channel index doesn't exist in the map, append 0
                evt_energies.append(0)
                continue

            # Mask for hits at this layer and this (x, y) position
            mask = (layers == layer) & (x_pos == xy_lookup[0]) & (y_pos == xy_lookup[1])
            
            if np.any(mask):
                # If there's a matching hit, get the energy (assume only one per channel)
                energy = energies[mask][0]
            else:
                energy = 0

            evt_energies.append(energy/0.000875127161931984)

    all_event_energies.append(evt_energies)



0.0
2.0
4.0
6.0
8.0
10.0
12.0
14.0
16.0
18.0
20.0
22.0
24.0
26.0
28.0
30.0
32.0
34.0
36.0
38.0
40.0
42.0
44.0
46.0
48.0
50.0
52.0
54.0
56.0
58.0
60.0
62.0
64.0
66.0
68.0
70.0
72.0
74.0
76.0
78.0
80.0
82.0
84.0
86.0
88.0
90.0
92.0
94.0
96.0
98.0


In [12]:
# Invert the channel maps: ch → (x, y)
even_ch_to_xy = {v: k for k, v in even_ch_map.items()}
odd_ch_to_xy  = {v: k for k, v in odd_ch_map.items()}

# Invert the z-layer map: layer → z
layer_to_z = {v: float(k) for k, v in z_to_layer_map.items()}

positions = []

for layer in range(15):
    ch_map = even_ch_to_xy if layer % 2 == 0 else odd_ch_to_xy

    for ch in range(25):
        if ch in ch_map and layer in layer_to_z:
            x, y = ch_map[ch]
            z = layer_to_z[layer]
        else:
            x, y, z = None, None, None
        #positions.append([x, y, z-5002.756, layer, ch])
        positions.append([x, y, z, layer, ch])


In [13]:
sim_data = {
    'positions': np.array(positions),
    'events': np.array(all_event_energies, dtype=object)
}

In [14]:
with open('../../genIII_data/Sim/e+_2.5_12.5.pkl', 'wb') as f:
    pickle.dump(sim_data, f)